# Hbase in Python
## Requirements
- Hbase Installed
- Start Hbase: ./start-hase.sh
    - This will require the Zookeeper running, or in standalone mode the conf file specifications
- Start the Hbase thrift server
    - ./hbase thrift start

## Util: Create Table: Creates Table if not already present

In [3]:
import happybase

hostName='localhost'
tableName='HBase-Test'
column_family_name='cerebral-cortex'
column_qualifier_name1='accx'
column_qualifier_name2='accy'
column_qualifier_name3='accz'

# Creating the Table in which we will insert the data
def CreateTable():
    connection = happybase.Connection(hostName)
    connection.open()
    tables = connection.tables()
    if tableName.encode() in tables :
        print('Table', tableName,'already exist')
    else:
        print(tables)
        connection.create_table(
             tableName,
             {
              column_family_name: dict(),  # use defaults
              }
             )
        print('Table',tableName, 'created')

# Util: Inserts Data

In [5]:
def InsertData(data):
    connection = happybase.Connection(hostName)
    connection.open()
    table = connection.table(tableName)
    column_name = '{fam}:{qual}'.format(fam=column_family_name,qual=column_qualifier_name1)
    row_key = 'row_key{}'.format(123)
    table.put(row_key, {column_name: data})